# このノートブックについて

### 概要
これまでの Tutorial では OpenAI Gym にプリインストールされている環境での訓練であったが、<br>
今回はオリジナルの環境を用いて強化学習を行う。<br>
CartPole を継承して作成された CartPoleCustom を環境として用いる。<br>
CartPoleCustom は元の CartPole に対して以下の点で異なる。

1. 台車と地面の間に摩擦がある
1. 台車と棒の間に摩擦がある

### 身につけられる知識
1. オリジナルの環境を用いた強化学習

# CartPole Custom について
<img src="./images/about_cartpolecustom_1.png" alt="about_cartpolecustom_1.png" />
<img src="./images/about_cartpolecustom_2.png" alt="about_cartpolecustom_1.png" />

In [ ]:
from distutils import dir_util
import os.path

# Azure Machine Learning core imports
import azureml.core
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace, Dataset
from azureml.core.compute import ComputeInstance
from azureml.core.compute_target import ComputeTargetException
from azureml.core.experiment import Experiment
from azureml.contrib.train.rl import ReinforcementLearningEstimator, Ray
from azureml.widgets import RunDetails

import consts

# Workspace と Compute target の設定
1つめのチュートリアルで説明済みなので、詳細は省略します

### Workspace の取得

In [ ]:
# Workspace の取得
interactive_auth = InteractiveLoginAuthentication(tenant_id=consts.tenant_id)

ws = Workspace(subscription_id=consts.subscription_id,
               resource_group=consts.resource_group,
               workspace_name=consts.workspace_name,
               auth=interactive_auth)
print(ws.name, ws.location, ws.resource_group, sep = ' | ')

### Computing cluster を compute target として指定する

**compute target** は訓練スクリプト、シミュレーションスクリプトを実行する際に用いられるコンピュータリソースです。<br>
compute target にはローカルコンピュータを指定することもできますし、クラウド上のコンピュータを指定することもできます。<br>
詳しくは [What are compute targets in Azure Machine Learning?](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target) を御覧ください。

このデモでは computing cluster を compute target として用います。<br>
`cc-dllab-d2v2`という名称の computing cluster を探し、なければこれを作成して使用します。

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
import os

# Cluster の名前とサイズを選択する
compute_name = consts.cc_cpu_name
compute_min_nodes = 0
compute_max_nodes = 4
vm_size = "STANDARD_D2_V2"

if compute_name in ws.compute_targets:
    print("次の Cluster が見つかりました: " + compute_name)
    compute_target = ws.compute_targets[compute_name]
else:
    print("新しい Cluster を作成します")
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size=vm_size,
        min_nodes=compute_min_nodes, 
        max_nodes=compute_max_nodes)
        
    # Cluster の作成
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

print(compute_target.get_status().serialize())

# 訓練

CartPoleCustom の訓練スクリプトは `files/cartpolecustom_training.py` となります。

`files/cartpolecustom_training.py` の中で独自の環境である `CartPoleCustom` が使われています。<br>
独自のクラスは `create_env` 関数を実装する必用があり、この関数への引数として下記の `env_config` が渡されることになります。

`files/cartpolecustom_training.py` への引数によって、カートと地面の摩擦、カートと棒の摩擦、棒の長さを設定することができます。<br>
この引数の指定が `script_params` 内の `env_config` で行われています。

In [ ]:
training_algorithm = "SAC"
rl_environment = "CartPoleCustom-v0"

script_params = {

    # 訓練アルゴリズム
    "--run": training_algorithm,  # "SAC"
    
    # 環境名
    "--env": rl_environment,  # "CartPoleCustom-v0"
    
    # 訓練に関係する config を設定する
    "--config": '\'{"num_workers": 1, "target_entropy": 0.1, "prioritized_replay": "", "env_config": {"friction_cart": 0.5, "friction_pole": 0.5, "pole_length": 1.25}}\'',  # 独自クラスのコンストラクタ引数
    
    # 訓練の終了条件
    # Simulationをスタート (リセット) してから終了するまでをエピソードという
    # ここでは複数回エピソードの平均報酬が 200 に達するか、訓練時間が 300秒を超えると訓練を終了する
    "--stop": '\'{"episode_reward_mean": 198}\'', 
    
    # チェックポイント (モデルの重みなど) の作成頻度
    # ここでは 2エピソード毎に作成する
    "--checkpoint-freq": 2,
    
    # 訓練終了時にもチェックポイントを作成する。値は空欄でOK。
    "--checkpoint-at-end": "",
    
    # Tensorboard で開くことのできるログの場所を指定する
    "--local-dir": './logs',
}

#  Reinforcement learning estimator
training_estimator = ReinforcementLearningEstimator(
    
    # 訓練スクリプトが入っているフォルダを指定
    source_directory='files',
    
    # 訓練スクリプトのファイル名
    entry_script="cartpolecustom_training.py",
    
    # 上で定義した訓練スクリプトへ渡す引数
    script_params=script_params,
     
    # compute target を指定する。ここではこのノートブックを開いている compute instance を指定する
    compute_target=compute_target,
            
    # 今現在は Ray() で固定
    rl_framework=Ray(),

    # pip を使ってライブラリを追加する。最新の ray を使用する。
    pip_packages=["ray[rllib]==0.8.7"]
)

In [ ]:
# 実験の作成
experiment_name = 'Demo03-CartPoleCustom-SAC-Train'  # 任意の名称を入力
exp = Experiment(workspace=ws, name=experiment_name)

# 訓練を実行する
training_run = exp.submit(training_estimator)

# 訓練をモニタリングする
RunDetails(training_run).show()

# 訓練完了を待つ
training_run.wait_for_completion()

### 子の実行へのハンドルを取得する
次の手順で子の実行へのハンドルを取得できます。<br>
本デモでは1つしか子の実行がないので、その `child_run_0` をそのハンドルとします。

In [ ]:
import time

child_run_0 = None
timeout = 30
while timeout > 0 and not child_run_0:
    child_runs = list(training_run.get_children())
    print('子の実行数:', len(child_runs))
    if len(child_runs) > 0:
        child_run_0 = child_runs[0]
        break
    time.sleep(2) # Wait for 2 seconds
    timeout -= 2

print('子の実行の情報:')
print(child_run_0)

# 訓練済みモデルで推論を行い、結果を確認する
RLlib が提供する補助スクリプトである `rollout.py` で訓練済みモデルを評価することができます。<br>
詳しくは [Evaluating Trained Policies](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies) を御覧ください。<br>
ここではこのスクリプトを用いて先程の訓練モデルを Machine learning work space に登録し、これを使用します。<br>
先程の訓練で `checkpoint-freq` と `checkpoint-at-end` を指定していればチェックポイントが生成されています。

ここではこれらのチェックポイントへのアクセス方法とそれを使って訓練済みモデルを評価する方法を示します。

### 訓練の生成物を取得する
訓練済みモデルを含む、訓練中の生成物は既定のデータストアに保存されています。

In [ ]:
from azureml.core import Dataset

run_id = child_run_0.id # 上で取得した子の実行の ID。親の実行ではないので注意
run_artifacts_path = os.path.join('azureml', run_id)
print("生成物のパス:", run_artifacts_path)

# デフォルトデータストアのファイルからデータセットオブジェクトを作成する
datastore = ws.get_default_datastore()
training_artifacts_ds = Dataset.File.from_files(datastore.path(os.path.join(run_artifacts_path, '**')))

確認のため、データセットに含まれるファイル数をプリントします。

In [ ]:
artifacts_paths = training_artifacts_ds.to_path()
print("データセットに含まれるファイルの数:", len(artifacts_paths))

# 以下をコメントアウトすると全てのファイルパスを print します
#print("生成物のパス: ", artifacts_paths)

### チェックポイントを取得する

In [ ]:
# チェックポイントの一覧を取得し、最後のものを使用する
checkpoint_files = [
    os.path.basename(file) for file in training_artifacts_ds.to_path() \
        if os.path.basename(file).startswith('checkpoint-') and \
            not os.path.basename(file).endswith('tune_metadata')
]

checkpoint_numbers = []
for file in checkpoint_files:
    checkpoint_numbers.append(int(file.split('-')[1]))

print("チェックポイント番号の一覧:", checkpoint_numbers)

last_checkpoint_number = max(checkpoint_numbers)
print("最後のチェックポイント番号:", last_checkpoint_number)

### 訓練済みモデルを登録する
Machine learning work space に最後に保存された訓練済みモデルを登録します。<br>
訓練目標達成時に保存されたモデルなので、最も性能が高いものであると仮定しています。<br>
登録されたモデルは再利用することができます。

In [ ]:
from azureml.core.model import Model
import tempfile

last_checkpoint_file = [file for file in training_artifacts_ds.to_path() \
        if os.path.basename(file).endswith(f"checkpoint-{last_checkpoint_number}")][0]
print("最後に記録されたチェックポイントファイル:", last_checkpoint_file)

last_checkpoint = os.path.dirname(os.path.join(run_artifacts_path, os.path.normpath(last_checkpoint_file[1:])))
print("最後に記録されたチェックポイント:", last_checkpoint)

model_name = consts.model_name_cartpolecustom_sac
model_path_prefix = os.path.join(tempfile.gettempdir(), 'tmp_training_artifacts')
datastore.download(target_path=model_path_prefix, prefix=last_checkpoint.replace('\\', '/'), show_progress=True)

# モデルの登録
model = Model.register(
    workspace=ws,
    model_path=os.path.join(model_path_prefix, last_checkpoint),
    model_name=model_name,
    description='CartpoleCustom 用に訓練された SAC モデル')

### 動画再生用環境の作成
Ray の動画録画機能を使うために xvfb 環境を構築します。
ここで作成した xvfb_env で推論が実行されます。

(1) Dockerファイルを使い、xvfb, ffmpeg, python-opengl, その他の依存をインストールします。<br>
注意: Renering off のときでも python-opengl は必用です。<br>
Docker ファイルは ./files/docker フォルダに入っています。<br>

(2) Headless display drivere をセットアップするため、xvfb-runコマンド経由で Python プロセスを実行します。

In [ ]:
from azureml.core.environment import Environment
video_capture = True

with open("files/docker/Dockerfile", "r") as f:
    dockerfile=f.read()

xvfb_env = Environment(name='xvfb-vdisplay')
xvfb_env.docker.enabled = True
xvfb_env.docker.base_image = None
xvfb_env.docker.base_dockerfile = dockerfile
    
xvfb_env.python.user_managed_dependencies = True
if video_capture:
    xvfb_env.python.interpreter_path = "xvfb-run -s '-screen 0 640x480x16 -ac +extension GLX +render' python"

### 推論の実行・動画の記録

In [ ]:
script_params = {    
    # 推論を実行する保存済のモデルの名前
    '--model_name': consts.model_name_cartpolecustom_sac,

    # 訓練アルゴリズム
    "--run": training_algorithm,  # "SAC"
    
    # 環境名
    "--env": rl_environment, # "CartPoleCustom-v0"
    
    # アルゴリズムパラメータ。訓練時と同じ摩擦と棒の長さを与えます。
    "--config": '\'{"env_config": {"friction_cart": 0.5, "friction_pole": 0.5, "pole_length": 1.25}}\'',  # 独自クラスのコンストラクタ引数
    
    # 推論を行うエピソード数。今回は5回なので、5個動画が作成される。
    "--episodes": 5,
        
    # 動画の保存先
    "--video-dir": "./logs/video"
}

rollout_estimator = ReinforcementLearningEstimator(
    # 推論スクリプトが入っているフォルダを指定
    source_directory='files',
    
    # 推論スクリプトのファイル名
    entry_script='cartpolecustom_rollout.py',
    
    # 上で定義した訓練スクリプトへ渡す引数
    # どのようにパースされるかは cartpole_training.py を参照
    script_params = script_params,
        
    # compute target を指定する。ここではこのノートブックを開いている compute instance を指定する
    compute_target=compute_target,
    
    # R今現在は Ray() で固定
    rl_framework=Ray(),
    
    # 動画記録用環境を指定する
    environment=xvfb_env
)

In [ ]:
# 実験の作成
experiment_name = 'Demo03-CartPoleCustom-SAC-Predict'  # 任意の名称を入力
exp = Experiment(workspace=ws, name=experiment_name)

# 訓練を実行する
rollout_run = exp.submit(rollout_estimator)

# 訓練をモニタリングする
RunDetails(rollout_run).show()

# 推論完了を待つ
rollout_run.wait_for_completion()

### 動画再生用の補助関数
ノートブック内で動画を再生するための補助関数を定義します。

In [ ]:
from distutils import dir_util
import shutil

# データセットからこのローカルに動画をダウンロードするための補助関数
def download_movies(artifacts_ds, movies, destination):
    # 動画の保存先を作成    
    if os.path.exists(destination):
        dir_util.remove_tree(destination)
    dir_util.mkpath(destination)

    for i, artifact in enumerate(artifacts_ds.to_path()):
        if artifact in movies:
            print('ダウンロード中 {} ...'.format(artifact))
            artifacts_ds.skip(i).take(1).download(target_path=destination, overwrite=True)

    print('動画のダウンロードが完了しました。')


# データセットのディレクトリ内の動画を探す補助関数
def find_movies(movie_path):
    print("動画を探すパス:", movie_path)
    mp4_movies = []
    for root, _, files in os.walk(movie_path):
        for name in files:
            if name.endswith('.mp4'):
                mp4_movies.append(os.path.join(root, name))
    print('{} 個の動画が見つかりました。'.format(len(mp4_movies)))

    return mp4_movies


# このノートブックに動画を描画する補助関数
from IPython.core.display import display, HTML
def display_movie(movie_file):
    display(
        HTML('\
            <video alt="cannot display video" autoplay loop> \
                <source src="{}" type="video/mp4"> \
            </video>'.format(movie_file)
        )
    )

### 動画の再生

動画を再生するためにまずローカルに動画をダウンロードします。<br>
ロールアウトの成果物のデータセットを作成し、先に定義した補助関数を使ってダウンロードを行い、動画を再生します。

In [ ]:
# 推論を実行した実行を取得する
child_runs = list(rollout_run.get_children())
print('子の実行数:', len(child_runs))
child_run_0 = child_runs[0]

run_id = child_run_0.id
run_artifacts_path = os.path.join('azureml', run_id)
print("実行の生成物へのパス:", run_artifacts_path)

# データセットオブジェクトの作成
datastore = ws.get_default_datastore()
rollout_artifacts_ds = Dataset.File.from_files(datastore.path(os.path.join(run_artifacts_path, '**')))

artifacts_paths = rollout_artifacts_ds.to_path()
print("データセット内のファイル数:", len(artifacts_paths))

# 一番最後に作成された動画を探す
mp4_files = [file for file in rollout_artifacts_ds.to_path() if file.endswith('.mp4')]
mp4_files.sort()

last_movie = mp4_files[-1] if len(mp4_files) > 1 else None
print("最後に保存された動画ファイル:", last_movie)

# 最後の動画をローカルにダウンロードする
rollout_movies_path = os.path.join("rollout", "videos")
download_movies(rollout_artifacts_ds, [last_movie], rollout_movies_path)

# ダウンロードされた動画を探す
mp4_files = find_movies(rollout_movies_path)
mp4_files.sort()

last_movie = mp4_files[-1] if len(mp4_files) > 0 else None
print("最後に保存された動画ファイル:", last_movie)

display_movie(last_movie) # 動画を描画する